# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [1]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'tensorflow-training-191120-1317'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [2]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

16 training jobs have completed


In [5]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2019, 11, 20, 13, 32, 44, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'f1_score',
                                                 'Value': 0.4595000147819519},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2019, 11, 20, 13, 37, 15, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:869530972998:training-job/tensorflow-training-191120-1317-009-61fc9499',
 'TrainingJobName': 'tensorflow-training-191120-1317-009-61fc9499',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2019, 11, 20, 13, 35, 21, tzinfo=tzlocal()),
 'TunedHyperParameters': {'batch-size': '16',
                          'dense-layer': '42',
                          'epochs': '9',
                          'layers': '1',
                          'learning-rate': '0.09608245871965426'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [6]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 16
{'lowest': 0.0, 'highest': 0.4595000147819519}


,FinalObjectiveValue,TrainingElapsedTimeSeconds,TrainingEndTime,TrainingJobName,TrainingJobStatus,TrainingStartTime,batch-size,dense-layer,epochs,layers,learning-rate
7,0.4595,114.0,2019-11-20 13:37:15+00:00,tensorflow-training-191120-1317-009-61fc9499,Completed,2019-11-20 13:35:21+00:00,16.0,42.0,9.0,1.0,0.096082
12,0.3861,64.0,2019-11-20 13:24:06+00:00,tensorflow-training-191120-1317-004-29b19529,Completed,2019-11-20 13:23:02+00:00,73.0,8.0,3.0,4.0,0.095437
5,0.3665,95.0,2019-11-20 13:40:28+00:00,tensorflow-training-191120-1317-011-e19ed8de,Completed,2019-11-20 13:38:53+00:00,67.0,6.0,3.0,4.0,0.094358
11,0.3591,97.0,2019-11-20 13:28:46+00:00,tensorflow-training-191120-1317-005-0dd5d101,Completed,2019-11-20 13:27:09+00:00,140.0,20.0,5.0,4.0,0.099499
1,0.3404,74.0,2019-11-20 13:48:35+00:00,tensorflow-training-191120-1317-015-a10233d4,Completed,2019-11-20 13:47:21+00:00,276.0,35.0,3.0,4.0,0.096902
13,0.3276,134.0,2019-11-20 13:24:56+00:00,tensorflow-training-191120-1317-003-6003542d,Completed,2019-11-20 13:22:42+00:00,76.0,54.0,3.0,4.0,0.034291
2,0.2681,54.0,2019-11-20 13:44:53+00:00,tensorflow-training-191120-1317-014-3a4cd553,Completed,2019-11-20 13:43:59+00:00,904.0,28.0,10.0,1.0,0.081131
6,0.2577,74.0,2019-11-20 13:36:19+00:00,tensorflow-training-191120-1317-010-884eb0ea,Completed,2019-11-20 13:35:05+00:00,264.0,15.0,3.0,1.0,0.091282
10,0.2205,61.0,2019-11-20 13:28:49+00:00,tensorflow-training-191120-1317-006-70133d41,Completed,2019-11-20 13:27:48+00:00,440.0,47.0,4.0,1.0,0.072678
14,0.2143,73.0,2019-11-20 13:20:44+00:00,tensorflow-training-191120-1317-002-92c8f6dc,Completed,2019-11-20 13:19:31+00:00,750.0,7.0,8.0,3.0,0.080813


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [8]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [11]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))